# Statistikkprosjekt

Vårt første modelleringsprosjekt er et prosjekt der du skal samle inn et datasett og gjøre statistisk analyse og regresjonsanalyse med disse dataene. Oppgaven går ut på følgende:

1. Samle inn data. Du bør måle minst 6 variabler og samle inn minst 25 målepunkter for hver av de fire variablene. Du kan bruke sensorer, spørreundersøkelser eller tilsvarende. Lagre dataene i en .txt-fil, .csv-fil eller .xlsx-fil (Excel). Hvis du finner et veldig godt datasett på internett som du heller ønsker å bruke, spør læreren om dere kan bruke det isteden.

2. Les dataene i Python og lag relevante visualiseringer som beskriver hva datasettet forteller oss.

3. Beregn relevante mål på sentraltendens og spredning. Presenter resultatene i en oversiktlig tabell. Forklar hva disse målene forteller om dataene dine. Lag gjerne visualiseringer som beskriver spredningen.

4. Regn ut korrelasjoner i datasettet og lag et korrelasjonsplott. Diskuter eventuelle korrelasjoner.

5. Gjør en lineær regresjon av to variabler der en lineær modell kan beskrive sammenhengen mellom variablene. Diskuter modellen.



Prosjektet leveres som fullstendig, sammensatt rapport (lever en .ipynb-fil og .pdf av notebooken). Husk å skrive navnet på alle i gruppa på rapporten. Alle på gruppa skal levere den samme fila på Teams.

## Resources
Link til dataset kan finnes [her](https://frost.met.no/observations/v0.csv?sources=sn18700&referencetime=R4/2022-08-05T06/2022-12-05T06/P1D&elements=air_temperature,min(air_temperature%20PT1H),max(air_temperature%20PT1H)&levels=default&timeoffsets=default&fields=sourceId,referenceTime,value,elementId)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import Api

In [2]:
df = Api.getData(reftime="<something>")

FileNotFoundError: [Errno 2] No such file or directory: 'kjoretoy.csv'

In [ ]:
df.head()

,tekn_unr,tekn_kjm_farge,tekn_reg_aar,tekn_f_g_n,tekn_avreg_dato,tekn_reg_status,tekn_bruktimp,tekn_merke,tekn_merkenavn,tekn_modell,...,tekn_siste_pkk,tekn_neste_pkk,tekn_standstoy,tekn_partikkelfilter,tekn_nox_utslipp_mgprkh,tekn_nox_utslipp_gprkwh,tekn_partikkelutslipp,tekn_maalemetode,tekn_co2_utslipp,tekn_drivstoff_forbruk
0,XMCLNDA1A3F003055,ORDINART,20021105,20021105,NaN,VRAKET,0.0,4011.0,MITSUBISHI,CARISMA,...,20140509.0,NaN,81.0,0.0,18,NaN,NaN,ANNEN,174,7.4
1,JSAERD31S00300834,ORDINART,20041230,20041230,NaN,REGISTRERT,0.0,8380.0,SUZUKI,LIANA,...,20200407.0,20220430.0,81.0,0.0,10,NaN,NaN,ANNEN,197,8.2
2,JHMAAD5430C116038,ORDINART,19851231,19850522,NaN,VRAKET,0.0,7740.0,HONDA,ACCORD,...,20010316.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,WV2ZZZ70ZSH075413,ORDINART,19950313,19950313,20161229.0,AVREGISTRERT,0.0,5750.0,VOLKSWAGEN,CARAVELLE,...,20151103.0,20170331.0,82.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,B364694,ORDINART,19800919,19800919,NaN,REGISTRERT,0.0,9600.0,SCHILTER,4600,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
